In [5]:
from random import randint, getrandbits
import itertools
import datetime
#import tensorflow.experimental.numpy as tnp
import timeit
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
from RunMe import *
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, GRU, Bidirectional, Activation, MaxPool1D, Conv1D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

%load_ext tensorboard
print(output_number)

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
10


## Input Parameters

In [6]:
file_paths ={
    'hla_a'  :0,
    #'hla_b'  :1,
    #'hla_c'  :2,
    'hla_dp' :3,
    'hla_dq' :4,
    'hla_dr' :5,
    #'hla_tap':6,
    #'hla_mica':7,
    #'hla_micb':8,
    'non-hla': 9
}
output_paths= {
    'hla_a'  :"/home/jovyan/data/Original/hla_a.fastm",
    #'hla_b'  :"/home/jovyan/data/Original/hla_b.fastm",
    #'hla_c'  :"/home/jovyan/data/Original/hla_c.fastm",
    'hla_dp' :"/home/jovyan/data/Original/hla_dp.fastm",
    'hla_dq' :"/home/jovyan/data/Original/hla_dq.fastm",
    'hla_dr' :"/home/jovyan/data/Original/hla_dr.fastm",
    #'hla_tap':"/home/jovyan/data/Original/hla_tap.fastm",
    #'hla_mica':"/home/jovyan/data/Original/hla_mica.fastm",
    #'hla_micb':"/home/jovyan/data/Original/hla_micb.fastm",
    #'sample'  :"data/Original/sample.fastm"
    'non-hla' :"/home/jovyan/data/Sample/refMrna.fastm"
}

training_params = {
    'num_samples': 1000,
    'batch_size': 100,
    'nb_epochs' : 2000,
    'learning_rate': 1e-4,
    'n_prob_upper_limit':.06,
    'mutation_prob_upper_limit':.02,
    'delete_prob_upper_limit':.015,
    'insert_prob_upper_limit':.015,
    'min_read_length':75,
    'b_randomize_location':True,
    'b_randomize_direction':False
}

output_classes = {
    'hla_a'  :0,
    #'hla_b'  :1,
    #'hla_c'  :2,
    'hla_dp' :3,
    'hla_dq' :4,
    'hla_dr' :5,
    #'hla_tap':6,
    #'hla_mica':7,
    #'hla_micb':8,
    'non-hla': 9
}


mp = {
        'max_read_length':150,
        'loss':'binary_crossentropy',
        'optimizer':'rmsprop',
        'metrics':['accuracy'],
        'num_classes': len(output_classes.keys())
    }


print(output_number) 
#print(X_test)

10


## Data Processing

In [7]:
y_train_correct = []
y_test_correct = []
y_data_correct = []
 

#for row in y_data: 
#    i = 0
#    for index in row:
#        if index == 1:
#            y_data_correct.append(i)
#        else:
#            i = i+1
#y_data_correct = np.array(y_data_correct)
print(x_data.shape)

X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3)

print(X_train.dtype)
print(y_train.dtype)
print(X_test.dtype)
print(y_test.dtype)

(12000, 150, 4)
int64
int64
int64
int64


## Working NN

In [9]:
n_features = X_train.shape[1]

model = tf.keras.Sequential()#[tf.keras.layers.Flatten(input_shape=(n_features, 4))]) 

model.add(tf.keras.layers.Conv1D(filters = 320, kernel_size = 20, 
                                strides=3, padding='valid', activation='relu', input_shape=(n_features,4)))

model.add(layers.MaxPool1D(pool_size=3, strides=None))

model.add(Dropout(0.5))

model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32), merge_mode = 'concat'))# merge_mode='concat', weights=None, backward_layer=None))

model.add(Dropout(0.5))

model.add(Dense(4, activation='softmax', kernel_initializer='he_normal'))

model.add(Dropout(0.5))

#model.add(Dense(1, activation='softmax', kernel_initializer='he_normal'))
model.add(Dense(output_number, activation='softmax', kernel_initializer='he_normal'))

optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)

#model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer=optimizer, loss='SparseCategoricalCrossentropy', metrics=['accuracy'])
#model.compile(optimizer=optimizer, loss='BinaryCrossentropy', metrics=['accuracy'])

model.summary()

model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=8, epochs=50, verbose = 1, shuffle = True)

loss, acc = model.evaluate(X_test, y_test, verbose=1)
print('Test Accuracy: %.3f' % acc)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_2 (Conv1D)            (None, 44, 320)           25920     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 14, 320)           0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 14, 320)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 64)                90368     
_________________________________________________________________
dropout_7 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 260       
_________________________________________________________________
dropout_8 (Dropout)          (None, 4)                

In [ ]:
outputs = []
for layer in model.layers:
    keras_function = tf.keras.backend.function([model.input], [layer.output])
    outputs.append(keras_function([X_train, 1]))
print(outputs)

#tf.argmax(input=n_features)

In [ ]:
#Link: https://www.machinecurve.com/index.php/2020/02/21/how-to-predict-new-samples-with-your-keras-model/
predictions = model.predict(X_test)

#Getting the max probabilities for the predictions
y_pred_classes = np.argmax(predictions, axis = 1)
#print(y_pred_classes)

result = tf.math.confusion_matrix(y_test,y_pred_classes)

 
# Printing the result
print('Confusion_matrix: ',result )
 

#print('Confusion Matrix\n')
#print(confusion)

classification_report(y_test_correct, y_pred_classes)
print('\nClassification Report\n')
print(classification_report(y_test, y_pred_classes, target_names=['non-hla', 'hla_a', 'hla_dp', 'hla_dq', 'hla_dr']))

## Grid Search

In [ ]:
## NN Parameters for tuning
#alphas = np.array([0.2,0.1,0.01,0.05,0.001])
#learningrateinit = np.array([0.2,0.1,0.01,0.001])
#hiddenlayers = np.array([50,100,200,300])
#maxiter = np.array([100,200,300])
#solvers = ('adam','lbfgs') #('lbfgs', 'sgd', 'adam')
#activations = ('relu') #,'logistic','tanh','identity')  # ('identity', 'logistic', 'tanh', 'relu')
#param = dict( activation=activations, solver=solvers, alpha = alphas, hidden_layer_sizes = hiddenlayers, max_iter = maxiter, learning_rate_init = learningrateinit)


#learningrateinit = np.array([0.1,0.01])
#hiddenlayers = np.array([50,100,200])
#param = dict(hidden_layer_sizes = hiddenlayers, learning_rate_init = learningrateinit)

#grid = GridSearchCV(estimator=model, param_grid=param,
#                    cv=5, verbose=10)
#grid = GridSearchCV(model,param, cv=5)
#grid.fit(X, y)


## Original NN Code

In [ ]:
'''
num_classes = len(output_classes.keys())
for train_class, idx in output_classes.items():
    output_arr = np.zeros(num_classes)
    output_arr[idx] = 1
    train_sampler = FastmSampler(fastm_file_path=output_paths[train_class],
                        n_prob_upper_limit=training_params['n_prob_upper_limit'],
                        mutation_prob_upper_limit=training_params['mutation_prob_upper_limit'],
                        delete_prob_upper_limit=training_params['delete_prob_upper_limit'],
                        insert_prob_upper_limit=training_params['insert_prob_upper_limit'],
                        min_read_length=training_params['min_read_length'],
                        tensor_length=mp['max_fragment_length'],
                        b_randomize_location=training_params['b_randomize_location'],
                        b_randomize_direction=training_params['b_randomize_direction'],
                        training_output=output_arr
                       )
    print(train_class,idx,output_arr)
    fastm_samplers.append(train_sampler)
'''

In [ ]:
'''
def shuffle_x_y_together(a, b):
    """
    Shuffles 2 numpy arrays together at the 1st dimension. Keeps them together, such that (Xi,Yi) => (Xj,Xj)
    :param a: Numpy Array
    :param b: Numpy Array
    :return: Shuffled A, Shuffled B
    """
    a_s = a.shape[0]
    b_s = b.shape[0]
    assert a_s == b_s, "Arrays must be of same size"
    i = np.argsort(np.random.permutation(a_s))
    return a[i, :, :], b[i, :]

shuffle_x_y_together(total_tensor_array, total_tensor_class)


def build_train_sets(data_samplers,
                    sample_size=1000,
                     test_size=.2):
    """
    Builds creates a train/test tensor for a round of training. 
    :param data_samplers: list sfseventools.FastmSampler
    :param sample_size: Samples per tensor, default 1000
    :param test_size: Test size as percentage of sample size. Default .2
    :return: x_train, y_train, x_test, y_test
    """
    length = len(data_samplers)
    test_length = int(length * test_size)

    x_train = np.concatenate([data_samplers[i].build_tensor(sample_size) 
                                                for i in range(length)])
    y_train = np.concatenate([data_samplers[i].build_output_tensor(sample_size) for i in range(length)])
    x_test = np.concatenate([data_samplers[i].build_tensor(sample_size) for i in range(test_length)])
    y_test = np.concatenate([data_samplers[i].build_output_tensor(sample_size) for i in range(test_length)])


    return x_train, y_train, x_test, y_test

build_train_sets(total_tensor_array, 1000, 0.2)

'''

In [ ]:
def print_log(file_path,epoch):
    f = open(file_path,'a')
    f.write('Trained epoch: '+ epoch +'\n') # python will convert \n to os.linesep
    f.close()

# Log Training
def train_log(file_path, epoch, accuracy):
    f = open(file_path,'a')
    f.write('Epoch '+str(epoch)+ ' loss/accuracy:'+ str(accuracy)+'\n')
    print('Epoch '+str(epoch)+ ' loss/accuracy:'+ str(accuracy)+'\n')# python will convert \n to os.linesep
    f.close()

In [ ]:
def train_network(model,
                  data_sets,
                  output_file, 
                  log_filepath,
                  training_params,
                  history,
                  save_weights = True
                  ):

    test_loss = []
    test_acc = []
    
    for i in range(training_params['nb_epochs']):
        X_train,y_train,X_test,y_test = build_train_sets(data_samplers=fastm_samplers,
                  sample_size = training_params['num_samples'])
        print('epoch: ' + str(i))
        

       
        model.fit(X_train, y_train, batch_size=training_params['batch_size'],
                  epochs=1,
                  verbose=0, shuffle=True, callbacks=[history])    # Manually save model
        loss_and_metrics = model.evaluate(X_test, y_test, batch_size=training_params['batch_size'])
        test_loss.append(loss_and_metrics[0])
        test_acc.append(loss_and_metrics[1])
        train_log(log_filepath,str(i),loss_and_metrics)

        if save_weights and i%10 ==0:
            print("Saving Model")
            model.save_weights(output_file,overwrite=True)


In [ ]:
train_network(model=model, 
              data_sets=fastm_samplers,
              output_file = "/root/Notebooks/test/keras_files/hla_weights.h5",
              log_filepath= "/root/Notebooks/test/keras_files/log_file.txt",
              training_params = training_params,
              history=history
             )

In [ ]:
with open("/root/Notebooks/test/keras_files/hla_network_schema.json", "w") as json_file:
    json_file.write(model.to_json())



In [ ]:
model.save_weights("/root/Notebooks/test/keras_files/hla_weights_200epoch.h5",overwrite=True)

In [ ]:
res_x = x[10001]
res_y = y[10001]
tmp_x, tmp_y = shuffle_x_y(x,y)
for idx, arr in enumerate(tmp_x):
    if (arr == res_x).all():
        print (res_y, tmp_y[idx],idx)

In [ ]:
a = np.concatenate([fastm_samplers[i].build_tensor(15) for i in range(len(fastm_samplers))])

In [ ]:
len(fastm_samplers)

In [ ]:
np.random.choice(100,3)

In [ ]:
from random import uniform
int(round(uniform(1,18), 0))

In [ ]:
np.zeros(len(output_classes.keys()))

In [ ]:
for k,v in output_classes.items():
    print(k,v)

In [ ]:
a = np.array([
    [[0, 0, 0, 0, 0],
    [ 27,  0,  0,  0, 0]], 
    
    [[1, 1, 1, 1, 1],
    [ 1,  18, 1 ,  1, 1]],
    
    [[2, 2, 2, 2, 2],
    [ 2,  2,  22,  2, 2]], 
    
    [[3, 3, 3, 3, 3],
    [ 3,  3,  3,  33, 3]],
    
    [[4, 4, 4, 4, 4],
    [ 4,  4,  4,  4, 44]], 
    
    [[5, 5, 5, 5, 5],
    [ 5,  1,  5,  5, 55]],
])
b = np.array([[0,0,0],
              [1,0,0],
              [0,2,0],
              [0,3,0],
              [0,0,4],
              [0,0,5]])

In [ ]:
a.shape

In [ ]:
b.shape

In [ ]:
s_a, s_b = shuffle_x_y(a,b)

In [ ]:
s_a

In [ ]:
s_b

In [ ]:
a

In [ ]:
t = np.array([[[  0.,   1.,   2.],
                  [  3.,   4.,   5.]],

                 [[  6.,   7.,   8.],
                  [  9.,  10.,  11.]],

                 [[ 12.,  13.,  14.],
                  [ 15.,  16.,  17.]]])

In [ ]:
t[[0,2,1],:,:]

In [ ]:
np.asarray([train_negative.training_output for i in range(15)])